In [50]:
!pip install xgboost


In [51]:
import numpy as np
import pandas as pd
import tweepy as tw
import re

from nltk import PorterStemmer

import nltk
from wordcloud import WordCloud, ImageColorGenerator
from PIL import Image
import urllib
import requests
from textblob import TextBlob


from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import f1_score

from xgboost import XGBClassifier

In [2]:
consumer_key = 'o6NyNihiacYxMzyfzQ1rGhnCc'
consumer_secret = 'b0bNtn82iaYgpdeoi9Ia8HncdwiOa9XWgY4l0lve9jn2srC4wX'

In [3]:
auth = tw.AppAuthHandler(consumer_key, consumer_secret)
api = tw.API(auth)


In [4]:
def remove_url(txt):
    return " ".join(re.sub(r"http\S+", "", txt).split())

In [5]:
search_term = "#trump -filter:retweets"

tweets = tw.Cursor(api.search, q = search_term, lang = "en", since="2021-01-01", tweet_mode ="extended").items(1000)

In [6]:
tweet_text = []

In [7]:
for tweet in tweets:
    tweet_text.append(tweet._json["full_text"])

In [8]:
len(tweet_text)

1000

In [9]:
tweet_text[1]

"@charliekirk11 Yo, ever notice how Twitter puts those special emojis right next to leftist hashtags? #BlackLivesMatter #MeToo etc?\n\nBut no special emojis next to conservative ones - #CPACTX #MAGA #Trump \n\nlike SCOTUS if you're going to trust them regulating speech"

In [10]:
tweets_no_url =[]

In [11]:
for tweet in tweet_text:
    text = remove_url(tweet)
    tweets_no_url.append(str(text))

In [12]:
type(tweets_no_url)

list

In [13]:
len(tweets_no_url)

1000

In [14]:
tweets_no_url[1]

"@charliekirk11 Yo, ever notice how Twitter puts those special emojis right next to leftist hashtags? #BlackLivesMatter #MeToo etc? But no special emojis next to conservative ones - #CPACTX #MAGA #Trump like SCOTUS if you're going to trust them regulating speech"

In [15]:
df = pd.DataFrame(tweets_no_url)

In [16]:
df

,0
0,Books on President #Trump and or #Melania are ...
1,"@charliekirk11 Yo, ever notice how Twitter put..."
2,Check out Al USVI's video! #TikTok don’t call ...
3,It’s odd all were fully jabbed-allegedly-and 3...
4,Random Trump - #randomtrump #trump #donaldtrum...
...,...
995,#Trump is AntiAmerican. #McCarthy #GOPSeditiou...
996,READ! Statement from President Donald J. #Trum...
997,@JennaEllisEsq @A1Policy #maga getting scammed...
998,#Trump is AntiAmerican. #McCarthy #GOPSeditiou...


In [17]:
df.columns = ['tweet']

In [18]:
df['tidy_tweets'] = df['tweet'].str.replace("[^a-zA-Z#]", " ")

In [19]:
df

,tweet,tidy_tweets
0,Books on President #Trump and or #Melania are ...,Books on President #Trump and or #Melania are ...
1,"@charliekirk11 Yo, ever notice how Twitter put...",charliekirk Yo ever notice how Twitter put...
2,Check out Al USVI's video! #TikTok don’t call ...,Check out Al USVI s video #TikTok don t call ...
3,It’s odd all were fully jabbed-allegedly-and 3...,It s odd all were fully jabbed allegedly and ...
4,Random Trump - #randomtrump #trump #donaldtrum...,Random Trump #randomtrump #trump #donaldtrum...
...,...,...
995,#Trump is AntiAmerican. #McCarthy #GOPSeditiou...,#Trump is AntiAmerican #McCarthy #GOPSeditiou...
996,READ! Statement from President Donald J. #Trum...,READ Statement from President Donald J #Trum...
997,@JennaEllisEsq @A1Policy #maga getting scammed...,JennaEllisEsq A Policy #maga getting scammed...
998,#Trump is AntiAmerican. #McCarthy #GOPSeditiou...,#Trump is AntiAmerican #McCarthy #GOPSeditiou...


In [20]:
# remove stopwords
df['tidy_tweets'] = df['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [21]:
df

,tweet,tidy_tweets
0,Books on President #Trump and or #Melania are ...,"Books President #Trump #Melania highly trash, ..."
1,"@charliekirk11 Yo, ever notice how Twitter put...",@charliekirk11 ever notice Twitter puts those ...
2,Check out Al USVI's video! #TikTok don’t call ...,Check USVI's video! #TikTok don’t call #Biden ...
3,It’s odd all were fully jabbed-allegedly-and 3...,It’s were fully jabbed-allegedly-and number ha...
4,Random Trump - #randomtrump #trump #donaldtrum...,Random Trump #randomtrump #trump #donaldtrump ...
...,...,...
995,#Trump is AntiAmerican. #McCarthy #GOPSeditiou...,#Trump AntiAmerican. #McCarthy #GOPSeditiousTr...
996,READ! Statement from President Donald J. #Trum...,READ! Statement from President Donald #Trump: ...
997,@JennaEllisEsq @A1Policy #maga getting scammed...,@JennaEllisEsq @A1Policy #maga getting scammed...
998,#Trump is AntiAmerican. #McCarthy #GOPSeditiou...,#Trump AntiAmerican. #McCarthy #GOPSeditiousTr...


In [22]:
# tokenize tweets
tokenized_tweets = df['tidy_tweets'].apply(lambda x: x.split())
tokenized_tweets.head()

0    [Books, President, #Trump, #Melania, highly, t...
1    [@charliekirk11, ever, notice, Twitter, puts, ...
2    [Check, USVI's, video!, #TikTok, don’t, call, ...
3    [It’s, were, fully, jabbed-allegedly-and, numb...
4    [Random, Trump, #randomtrump, #trump, #donaldt...
Name: tidy_tweets, dtype: object

In [23]:
ps = PorterStemmer()
stemmed_tokenized_tweets = tokenized_tweets.apply(lambda words: [ps.stem(word) for word in words])

In [24]:
for i in range(len(stemmed_tokenized_tweets)):
    stemmed_tokenized_tweets[i] = ' '.join(stemmed_tokenized_tweets[i])
    
df['tidy_tweets'] = stemmed_tokenized_tweets
df

,tweet,tidy_tweets
0,Books on President #Trump and or #Melania are ...,"book presid #trump #melania highli trash, bias..."
1,"@charliekirk11 Yo, ever notice how Twitter put...",@charliekirk11 ever notic twitter put those sp...
2,Check out Al USVI's video! #TikTok don’t call ...,check usvi' video! #tiktok don’t call #biden c...
3,It’s odd all were fully jabbed-allegedly-and 3...,it’ were fulli jabbed-allegedly-and number hav...
4,Random Trump - #randomtrump #trump #donaldtrum...,random trump #randomtrump #trump #donaldtrump ...
...,...,...
995,#Trump is AntiAmerican. #McCarthy #GOPSeditiou...,#trump antiamerican. #mccarthi #gopseditioustr...
996,READ! Statement from President Donald J. #Trum...,"read! statement from presid donald #trump: ""@f..."
997,@JennaEllisEsq @A1Policy #maga getting scammed...,@jennaellisesq @a1polici #maga get scammed. pl...
998,#Trump is AntiAmerican. #McCarthy #GOPSeditiou...,#trump antiamerican. #mccarthi #gopseditioustr...


In [25]:

sia = SentimentIntensityAnalyzer()

In [26]:
def word_polarizer(text):
    sentiment_score = sia.polarity_scores(text)
    if sentiment_score['compound'] > 0:
        return 0
    elif sentiment_score['compound'] < 0:
        return 1
    else:
        return 0 # can be changed to add neutral
    

In [27]:
df['labels'] = df.apply(lambda row: word_polarizer(row['tidy_tweets']), axis=1)

In [28]:
df.labels.value_counts()

0    651
1    349
Name: labels, dtype: int64

In [29]:
df

,tweet,tidy_tweets,labels
0,Books on President #Trump and or #Melania are ...,"book presid #trump #melania highli trash, bias...",0
1,"@charliekirk11 Yo, ever notice how Twitter put...",@charliekirk11 ever notic twitter put those sp...,0
2,Check out Al USVI's video! #TikTok don’t call ...,check usvi' video! #tiktok don’t call #biden c...,1
3,It’s odd all were fully jabbed-allegedly-and 3...,it’ were fulli jabbed-allegedly-and number hav...,0
4,Random Trump - #randomtrump #trump #donaldtrum...,random trump #randomtrump #trump #donaldtrump ...,0
...,...,...,...
995,#Trump is AntiAmerican. #McCarthy #GOPSeditiou...,#trump antiamerican. #mccarthi #gopseditioustr...,0
996,READ! Statement from President Donald J. #Trum...,"read! statement from presid donald #trump: ""@f...",1
997,@JennaEllisEsq @A1Policy #maga getting scammed...,@jennaellisesq @a1polici #maga get scammed. pl...,0
998,#Trump is AntiAmerican. #McCarthy #GOPSeditiou...,#trump antiamerican. #mccarthi #gopseditioustr...,0


In [30]:
def Hashtags_Extract(x):
    hashtags=[]
    
    # Loop over the words in the tweet
    for i in x:
        ht = re.findall(r'#(\w+)',i)
        hashtags.append(ht)
    
    return hashtags

In [31]:
ht_positive = Hashtags_Extract(df['tidy_tweets'][df['labels']==0])

In [32]:
ht_negative = Hashtags_Extract(df['tidy_tweets'][df['labels']==1])

In [33]:
# Unnest the lists
ht_positive_unnest = sum(ht_positive,[])
ht_negative_unnest = sum(ht_negative,[])


In [34]:
word_freq_positive = nltk.FreqDist(ht_positive_unnest)

word_freq_positive

FreqDist({'trump': 666, 'gif': 174, 'donaldtrump': 99, 'biden': 96, 'maga': 93, 'randomtrump': 87, 'giphi': 87, 'gop': 37, 'usa': 32, 'republican': 28, ...})

In [35]:
word_freq_positive = nltk.FreqDist(ht_negative_unnest)

word_freq_positive

FreqDist({'trump': 362, 'ettd': 48, 'gop': 35, 'biden': 33, 'maga': 32, 'republican': 16, 'usa': 14, 'polit': 13, 'covid19': 12, 'new': 11, ...})

In [36]:
# Implementing Bag of words

bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2,  stop_words='english')

# feature matrix bow
bow = bow_vectorizer.fit_transform(df['tidy_tweets'])

df_bow = pd.DataFrame(bow.todense())
df_bow

,0,1,2,3,4,5,6,7,8,9,...,1623,1624,1625,1626,1627,1628,1629,1630,1631,1632
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
tfidf=TfidfVectorizer(max_df=0.90, min_df=2,stop_words='english')

tfidf_matrix=tfidf.fit_transform(df['tidy_tweets'])

df_tfidf = pd.DataFrame(tfidf_matrix.todense())

df_tfidf

,0,1,2,3,4,5,6,7,8,9,...,1623,1624,1625,1626,1627,1628,1629,1630,1631,1632
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
train_bow = bow.todense()

In [39]:
train_tfidf_matrix = tfidf_matrix.todense()

In [40]:
x_train_bow,x_valid_bow,y_train_bow,y_valid_bow = train_test_split(train_bow,df['labels'],test_size=0.3,random_state=2)

In [41]:
x_train_tfidf,x_valid_tfidf,y_train_tfidf,y_valid_tfidf = train_test_split(train_tfidf_matrix,df['labels'],test_size=0.3,random_state=17)

In [42]:
Log_Reg = LogisticRegression(random_state = 0, solver = "lbfgs")

In [43]:
Log_Reg.fit(x_train_bow, y_train_bow)

LogisticRegression(random_state=0)

In [44]:
prediction_bow =Log_Reg.predict_proba(x_valid_bow)
prediction_bow

array([[0.78629986, 0.21370014],
       [0.19212057, 0.80787943],
       [0.99196156, 0.00803844],
       [0.32063646, 0.67936354],
       [0.00551992, 0.99448008],
       [0.51534863, 0.48465137],
       [0.92110629, 0.07889371],
       [0.23058386, 0.76941614],
       [0.77176917, 0.22823083],
       [0.99196156, 0.00803844],
       [0.61462925, 0.38537075],
       [0.63895827, 0.36104173],
       [0.65737339, 0.34262661],
       [0.58718694, 0.41281306],
       [0.82019855, 0.17980145],
       [0.8625187 , 0.1374813 ],
       [0.20178422, 0.79821578],
       [0.96432217, 0.03567783],
       [0.96588799, 0.03411201],
       [0.11058686, 0.88941314],
       [0.60952662, 0.39047338],
       [0.99196156, 0.00803844],
       [0.73807952, 0.26192048],
       [0.73956245, 0.26043755],
       [0.6886312 , 0.3113688 ],
       [0.99196156, 0.00803844],
       [0.7062554 , 0.2937446 ],
       [0.79245178, 0.20754822],
       [0.17927428, 0.82072572],
       [0.7916884 , 0.2083116 ],
       [0.

In [45]:
prediction_int = prediction_bow[:,1]>=0.3

prediction_int = prediction_int.astype(np.int)
prediction_int

log_bow = f1_score(y_valid_bow, prediction_int)

log_bow

0.6173913043478261

In [46]:
# For tf idf features
Log_Reg.fit(x_train_tfidf,y_train_tfidf)

LogisticRegression(random_state=0)

In [47]:
prediction_tfidf = Log_Reg.predict_proba(x_valid_tfidf)
prediction_tfidf

array([[0.51835512, 0.48164488],
       [0.62510873, 0.37489127],
       [0.68393558, 0.31606442],
       [0.64810833, 0.35189167],
       [0.64199472, 0.35800528],
       [0.74747214, 0.25252786],
       [0.73203791, 0.26796209],
       [0.96180213, 0.03819787],
       [0.59443779, 0.40556221],
       [0.74051083, 0.25948917],
       [0.64601326, 0.35398674],
       [0.61650781, 0.38349219],
       [0.71995102, 0.28004898],
       [0.2435373 , 0.7564627 ],
       [0.48999159, 0.51000841],
       [0.67902871, 0.32097129],
       [0.69032809, 0.30967191],
       [0.64884145, 0.35115855],
       [0.58544674, 0.41455326],
       [0.71844794, 0.28155206],
       [0.5828279 , 0.4171721 ],
       [0.65051174, 0.34948826],
       [0.75633835, 0.24366165],
       [0.71099047, 0.28900953],
       [0.2658758 , 0.7341242 ],
       [0.85001597, 0.14998403],
       [0.75875168, 0.24124832],
       [0.67570427, 0.32429573],
       [0.68958022, 0.31041978],
       [0.6904365 , 0.3095635 ],
       [0.

In [48]:

prediction_int = prediction_tfidf[:,1]>=0.3

prediction_int = prediction_int.astype(np.int)
prediction_int

# calculating f1 score
log_tfidf = f1_score(y_valid_tfidf, prediction_int)

log_tfidf

0.6794871794871794

In [52]:
# Using XGBoost

In [53]:
model_bow = XGBClassifier(random_state=22,learning_rate=0.9)

In [54]:
model_bow.fit(x_train_bow, y_train_bow)


C:\Users\umang\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:47:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.9, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=22,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [55]:
xgb=model_bow.predict_proba(x_valid_bow)

xgb

array([[0.77547693, 0.22452308],
       [0.5446937 , 0.45530626],
       [0.99637455, 0.00362542],
       [0.05790764, 0.94209236],
       [0.00286615, 0.99713385],
       [0.577406  , 0.422594  ],
       [0.8169931 , 0.18300688],
       [0.57679427, 0.4232057 ],
       [0.95371395, 0.04628607],
       [0.99637455, 0.00362542],
       [0.69378555, 0.30621448],
       [0.35673708, 0.6432629 ],
       [0.18186623, 0.8181338 ],
       [0.15654159, 0.8434584 ],
       [0.44090217, 0.5590978 ],
       [0.960555  , 0.03944501],
       [0.09421933, 0.9057807 ],
       [0.8902943 , 0.1097057 ],
       [0.9640047 , 0.03599533],
       [0.03877914, 0.96122086],
       [0.47509748, 0.5249025 ],
       [0.99637455, 0.00362542],
       [0.77547693, 0.22452308],
       [0.88866854, 0.11133149],
       [0.8123061 , 0.1876939 ],
       [0.99637455, 0.00362542],
       [0.9021628 , 0.09783724],
       [0.59526205, 0.40473792],
       [0.0118764 , 0.9881236 ],
       [0.734477  , 0.26552302],
       [0.

In [56]:
xgb=xgb[:,1]>=0.3

# converting the results to integer type
xgb_int=xgb.astype(np.int)

# calculating f1 score
xgb_bow=f1_score(y_valid_bow,xgb_int)

xgb_bow

0.5658536585365854

In [57]:
# using tfidf 
model_tfidf=XGBClassifier(random_state=29,learning_rate=0.7)


In [58]:
model_tfidf.fit(x_train_tfidf, y_train_tfidf)

C:\Users\umang\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:48:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.7, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=29,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [59]:
xgb_tfidf=model_tfidf.predict_proba(x_valid_tfidf)

xgb_tfidf

array([[3.87467861e-01, 6.12532139e-01],
       [8.14200401e-01, 1.85799599e-01],
       [3.62646997e-01, 6.37353003e-01],
       [8.39979768e-01, 1.60020202e-01],
       [2.16626048e-01, 7.83373952e-01],
       [9.29228485e-01, 7.07715005e-02],
       [5.95275283e-01, 4.04724717e-01],
       [9.93474483e-01, 6.52553746e-03],
       [7.49033093e-01, 2.50966877e-01],
       [9.52630043e-01, 4.73699793e-02],
       [9.78739619e-01, 2.12603621e-02],
       [8.45871925e-01, 1.54128045e-01],
       [8.56959343e-01, 1.43040642e-01],
       [7.99357891e-04, 9.99200642e-01],
       [9.86477375e-01, 1.35226110e-02],
       [9.72365737e-01, 2.76342891e-02],
       [8.14200401e-01, 1.85799599e-01],
       [9.46964443e-01, 5.30355684e-02],
       [6.33843541e-02, 9.36615646e-01],
       [9.67905939e-01, 3.20940539e-02],
       [9.19698477e-01, 8.03015083e-02],
       [9.35077786e-01, 6.49222061e-02],
       [9.44715261e-01, 5.52847311e-02],
       [8.63947690e-01, 1.36052325e-01],
       [7.993578

In [60]:
xgb_tfidf=xgb_tfidf[:,1]>=0.3

# converting the results to integer type
xgb_int_tfidf=xgb_tfidf.astype(np.int)

# calculating f1 score
score=f1_score(y_valid_tfidf,xgb_int_tfidf)

score

0.5751072961373391